In [ ]:
pip install numpy pandas matplotlib

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib
# control the working directory
print(os.getcwd())

# data import and pandas manipulation
fp_savi = "../input.savi.txt"
cutoff_refdepth_B = 20
cutoff_altdepth_B = 1
cutoff_freq = 5

raw_data = pd.read_csv(fp_savi, delimiter="\t")
#print(raw_data.describe())

df_savi = raw_data.loc[lambda df:(df["refdepth_Blood"] >= cutoff_refdepth_B) & 
                                 (df["altdepth_Blood"] <= cutoff_altdepth_B) & 
                                 (df["Sgt1_max_frequency"] >= cutoff_freq), :]
df_savi = df_savi.fillna(0)


     chr        pos ref alt                               Effect  \
0  chr10    5248362   T   C  splice_donor_variant+intron_variant   
1  chr10   27703023   A   T                     missense_variant   
2  chr10  122263428   G   A                     missense_variant   
3  chr11     209598   C   T                   synonymous_variant   
4  chr11   20676292  TG   T                   frameshift_variant   

  Effect_Impact Functional_Class Codon_Change Amino_Acid_Change  \
0          HIGH                -            -                 -   
1      MODERATE         MISSENSE      Tcc/Acc              S53T   
2      MODERATE         MISSENSE      cGc/cAc              R52H   
3           LOW           SILENT      tcC/tcT              S108   
4          HIGH                -         tgg/              W758   

  Amino_Acid_length  ... Recurrent_freq  refdepth_Blood  altdepth_Blood  \
0               323  ...             20             229               0   
1               767  ...              